In [1]:
library(tidyverse)
library(cansim)
library(readr)
library(vroom)
library(lubridate)
library(tseries)
library(strucchange)
library(purrr) 
library(vars)
library(fredr)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.2     ✔ tibble    3.3.0
✔ lubridate 1.9.4     ✔ tidyr     1.3.1
✔ purrr     1.1.0     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors

Attaching package: 'vroom'


The following objects are masked from 'package:readr':

    as.col_spec, col_character, col_date, col_datetime, col_double,
    col_factor, col_guess, col_integer, col_logical, col_number,
    col_skip, col_time, cols, cols_condense, cols_only, date_names,
    date_names_lang, date_names_langs, default_locale, fwf_cols,
    fwf_empty, fwf_positions, fwf_widths, locale, output_column,
    problems, spec


Registered S3 method overwritten by 'q

In [2]:
fredr_set_key('b43b23bac4c602369428a1a4316f23d4')

In [3]:
all_tarrif_data <- read_csv("C:/Users/Irene/governorschallenge/DataWeb-Query-Export.csv")

general_import_charges <- all_tarrif_data|>
filter(`Data Type` == "General Import Charges")|>
dplyr::select(Year, Month, `General Customs Value`)|>
rename(VALUE = `General Customs Value`)|>
mutate(
    Date = ymd(paste(Year, Month, "01", sep = "-")),
    VALUE = as.numeric(VALUE)
  ) |>
  dplyr::select(Date, VALUE)|>
  arrange(Date)|>
  glimpse()

general_customs_value <- all_tarrif_data|>
filter(`Data Type` == "General Customs Value")|>
dplyr::select(Year, Month, `General Customs Value`)|>
rename(VALUE = `General Customs Value`)|>
mutate(
    Date = ymd(paste(Year, Month, "01", sep = "-")),
    VALUE = as.numeric(VALUE)
  ) |>
  dplyr::select(Date, VALUE)|>
  arrange(Date)|>
  glimpse()



tarrif_rate_df <- inner_join(
  general_import_charges, 
  general_customs_value, 
  by = "Date",
  suffix = c("_charges", "_value")
) |>
  
  # 2. Calculate the approximate tariff rate using the two VALUE columns
  mutate(
    Approximate_Rate = (VALUE_charges / VALUE_value) * 100
  ) |>
  rename(month = Date)|>
  # 3. Keep only the columns you need for your VAR model
  dplyr::select(month, Approximate_Rate)

# --- Check the final result ---
glimpse(tarrif_rate_df)

Rows: 616 Columns: 4
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (4): Data Type, Year, Month, General Customs Value

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


Rows: 306
Columns: 2
$ Date  <date> 2000-01-01, 2000-02-01, 2000-03-01, 2000-04-01, 2000-05-01, 200…
$ VALUE <dbl> 269869762, 278547963, 305301730, 290530227, 301763964, 297764412…
Rows: 306
Columns: 2
$ Date  <date> 2000-01-01, 2000-02-01, 2000-03-01, 2000-04-01, 2000-05-01, 200…
$ VALUE <dbl> 17705142820, 18087403941, 20618285791, 18506995213, 19558432594,…
Rows: 306
Columns: 2
$ month            <date> 2000-01-01, 2000-02-01, 2000-03-01, 2000-04-01, 2000…
$ Approximate_Rate <dbl> 1.524245, 1.540011, 1.480733, 1.569840, 1.542884, 1.4…


In [4]:
cpi_vectors <- c(
  "Food" = "v41690915",
  "Shelter" = "v41690916",
  "Household_operations" = "v41690917",
  "Clothing_footwear" = "v41690918",
  "Transportation" = "v41690919",
  "Health_personal_care" = "v41690920",
  "Recreation_education" = "v41690921",
  "Alcohol_tobacco" = "v41690922"
)

# Define the official 2023 CPI basket weights (from StatCan Table 18-10-0007-01)
# These weights are based on the 2022 Survey of Household Spending.
cpi_weights <- c(
  "Food" = 16.59,
  "Shelter" = 29.85,
  "Household_operations" = 13.52,
  "Clothing_footwear" = 4.49,
  "Transportation" = 16.14,
  "Health_personal_care" = 5.17,
  "Recreation_education" = 9.69,
  "Alcohol_tobacco" = 4.55
)

In [25]:
library(cansim)
library(dplyr)

# 1. Define a named vector with the CANSIM vector IDs for the CPI weights
# These are extracted from the data you sent.
cpi_weight_vectors <- c(
  Food = "v91858740",
  Shelter = "v91858892",
  Household_operations = "v91858926",
  Clothing_footwear = "v91859016",
  Transportation = "v91859056",
  Health_personal_care = "v91859106",
  Recreation_education = "v91859144",
  Alcohol_tobacco = "v91859220",
  Energy = "v91859272",
  Goods = "v91859278",
  Services = "v91873252"
)

# 2. Download the data for each component in the specified format
food_w_df <- get_cansim_vector(cpi_weight_vectors["Food"], start_time = "1996-01-01") %>% mutate(Component = "Food")
shelter_w_df <- get_cansim_vector(cpi_weight_vectors["Shelter"], start_time = "1996-01-01") %>% mutate(Component = "Shelter")
household_w_df <- get_cansim_vector(cpi_weight_vectors["Household_operations"], start_time = "1996-01-01") %>% mutate(Component = "Household_operations")
clothing_w_df <- get_cansim_vector(cpi_weight_vectors["Clothing_footwear"], start_time = "1996-01-01") %>% mutate(Component = "Clothing_footwear")
transport_w_df <- get_cansim_vector(cpi_weight_vectors["Transportation"], start_time = "1996-01-01") %>% mutate(Component = "Transportation")
health_w_df <- get_cansim_vector(cpi_weight_vectors["Health_personal_care"], start_time = "1996-01-01") %>% mutate(Component = "Health_personal_care")
recreation_w_df <- get_cansim_vector(cpi_weight_vectors["Recreation_education"], start_time = "1996-01-01") %>% mutate(Component = "Recreation_education")
alcohol_w_df <- get_cansim_vector(cpi_weight_vectors["Alcohol_tobacco"], start_time = "1996-01-01") %>% mutate(Component = "Alcohol_tobacco")
energy_w_df <- get_cansim_vector(cpi_weight_vectors["Energy"], start_time = "1996-01-01") %>% mutate(Component = "Energy")
goods_w_df <- get_cansim_vector(cpi_weight_vectors["Goods"], start_time = "1996-01-01") %>% mutate(Component = "Goods")
services_w_df <- get_cansim_vector(cpi_weight_vectors["Services"], start_time = "1996-01-01") %>% mutate(Component = "Services")

# Optional: Combine into a single dataframe
all_weights_df <- bind_rows(
  food_w_df, shelter_w_df, household_w_df, clothing_w_df, transport_w_df,
  health_w_df, recreation_w_df, alcohol_w_df, energy_w_df, goods_w_df, services_w_df
)

# View the combined result
glimpse(all_weights_df)

Warning message in as.POSIXlt.POSIXct(x, tz = tz):
"unknown timezone 'America/Toronto'"
Warning message in as.POSIXlt.POSIXct(x, tz = tz):
"unknown timezone 'GMT'"
Warning message in as.POSIXlt.POSIXct(x, tz = tz):
"unknown timezone 'America/New_York'"
Warning message in as.POSIXlt.POSIXct(x, tz = tz):
"unknown timezone 'America/Los_Angeles'"
Warning message in as.POSIXlt.POSIXct(x, tz = tz):
"unknown timezone 'GMT'"
Warning message in as.POSIXlt.POSIXct(x, tz = tz):
"unknown timezone 'America/New_York'"
Accessing CANSIM NDM vectors from Statistics Canada

Warning message in as.POSIXlt.POSIXct(x, tz = tz):
"unknown timezone 'America/Toronto'"
Warning message in as.POSIXlt.POSIXct(x, tz = tz):
"unknown timezone 'GMT'"
Warning message in as.POSIXlt.POSIXct(x, tz = tz):
"unknown timezone 'America/New_York'"
Warning message in as.POSIXlt.POSIXct(x, tz = tz):
"unknown timezone 'America/Los_Angeles'"
Warning message in as.POSIXlt.POSIXct(x, tz = tz):
"unknown timezone 'GMT'"
Warning message 

Rows: 143
Columns: 18
$ REF_DATE                            <chr> "1996-01-01", "2001-01-01", "2005-…
$ Date                                <date> 1996-01-01, 2001-01-01, 2005-01-0…
$ GEO                                 <fct> "Canada", "Canada", "Canada", "Can…
$ `Products and product groups`       <fct> "Food", "Food", "Food", "Food", "F…
$ `Price period of weight`            <fct> Weight at basket link month prices…
$ `Geographic distribution of weight` <fct> Distribution to selected geographi…
$ label                               <chr> "Food", "Food", "Food", "Food", "F…
$ VALUE                               <dbl> 17.89, 16.89, 17.04, 15.99, 16.60,…
$ val_norm                            <dbl> 17.89, 16.89, 17.04, 15.99, 16.60,…
$ SCALAR_ID                           <int> 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0…
$ VECTOR                              <chr> "v91858740", "v91858740", "v918587…
$ cansimTableNumber                   <chr> "18-10-0007", "18-10-0007", "18-10…
$ COORDINATE      

In [28]:
wide_weights_df <- all_weights_df %>%
  # The 'Date' column from cansim is already correctly formatted
  rename(date = Date) %>% 
  # Select only the necessary columns
  dplyr::select(date, Component, VALUE) %>%
  # Pivot!
  pivot_wider(
    names_from = Component,
    values_from = VALUE
  ) %>%
  # The weights are annual but reported monthly; fill downwards to apply for the whole year
  fill(everything(), .direction = "down") %>%
  # StatCan often reports weights in the month before they apply, so we lag them
  # This aligns the 2023 basket weight with the year 2023, for example.
  mutate(across(-date, ~lag(.x))) %>%
  drop_na() # Remove any rows with NAs after lagging

# View the result - it's now ready to be used
glimpse(wide_weights_df)

Rows: 12
Columns: 12
$ date                 <date> 2001-01-01, 2005-01-01, 2009-01-01, 2011-01-01, …
$ Food                 <dbl> 17.89, 16.89, 17.04, 15.99, 16.60, 16.41, 16.23, …
$ Shelter              <dbl> 26.75, 26.75, 26.62, 27.49, 26.26, 26.80, 27.15, …
$ Household_operations <dbl> 10.76, 10.58, 11.10, 11.55, 12.66, 13.14, 12.97, …
$ Clothing_footwear    <dbl> 6.25, 5.37, 5.36, 5.31, 5.82, 6.08, 5.44, 5.17, 3…
$ Transportation       <dbl> 18.96, 19.79, 19.88, 20.60, 19.98, 19.10, 19.70, …
$ Health_personal_care <dbl> 4.60, 4.52, 4.73, 4.95, 4.93, 4.73, 5.00, 4.79, 4…
$ Recreation_education <dbl> 11.25, 11.96, 12.20, 11.20, 10.96, 10.89, 10.89, …
$ Alcohol_tobacco      <dbl> 3.54, 4.13, 3.07, 2.91, 2.79, 2.86, 2.63, 3.21, 4…
$ Energy               <dbl> 8.38, 8.84, 9.38, 10.08, 8.56, 7.79, 7.06, 6.50, …
$ Goods                <dbl> 50.28, 48.84, 48.78, 47.80, 48.18, 46.68, 45.32, …
$ Services             <dbl> 49.72, 51.16, 51.22, 52.20, 51.82, 53.32, 54.68, …


In [29]:
wide_weights_df <- wide_weights_df %>%
  mutate(date = ymd(date))

# Complete the dataframe to have a row for every year and fill the values down
yearly_weights_df <- wide_weights_df %>%
  # Create rows for every year from the min to the max year in the data
  complete(date = seq.Date(min(date), max(date), by = "year")) %>%
  
  # Fill the NA values in all columns (except 'date') with the last known value
  fill(everything(), .direction = "down")

# View the result
# You'll see that you now have a row for 2001, 2002, 2003, 2004, etc.
glimpse(yearly_weights_df)

Rows: 24
Columns: 12
$ date                 <date> 2001-01-01, 2002-01-01, 2003-01-01, 2004-01-01, …
$ Food                 <dbl> 17.89, 17.89, 17.89, 17.89, 16.89, 16.89, 16.89, …
$ Shelter              <dbl> 26.75, 26.75, 26.75, 26.75, 26.75, 26.75, 26.75, …
$ Household_operations <dbl> 10.76, 10.76, 10.76, 10.76, 10.58, 10.58, 10.58, …
$ Clothing_footwear    <dbl> 6.25, 6.25, 6.25, 6.25, 5.37, 5.37, 5.37, 5.37, 5…
$ Transportation       <dbl> 18.96, 18.96, 18.96, 18.96, 19.79, 19.79, 19.79, …
$ Health_personal_care <dbl> 4.60, 4.60, 4.60, 4.60, 4.52, 4.52, 4.52, 4.52, 4…
$ Recreation_education <dbl> 11.25, 11.25, 11.25, 11.25, 11.96, 11.96, 11.96, …
$ Alcohol_tobacco      <dbl> 3.54, 3.54, 3.54, 3.54, 4.13, 4.13, 4.13, 4.13, 3…
$ Energy               <dbl> 8.38, 8.38, 8.38, 8.38, 8.84, 8.84, 8.84, 8.84, 9…
$ Goods                <dbl> 50.28, 50.28, 50.28, 50.28, 48.84, 48.84, 48.84, …
$ Services             <dbl> 49.72, 49.72, 49.72, 49.72, 51.16, 51.16, 51.16, …


In [7]:
food_df <- get_cansim_vector(cpi_vectors["Food"], start_time = "2000-01-01") %>% mutate(Component = "Food")
shelter_df <- get_cansim_vector(cpi_vectors["Shelter"], start_time = "2000-01-01") %>% mutate(Component = "Shelter")
household_df <- get_cansim_vector(cpi_vectors["Household_operations"], start_time = "2000-01-01") %>% mutate(Component = "Household_operations")
clothing_df <- get_cansim_vector(cpi_vectors["Clothing_footwear"], start_time = "2000-01-01") %>% mutate(Component = "Clothing_footwear")
transport_df <- get_cansim_vector(cpi_vectors["Transportation"], start_time = "2000-01-01") %>% mutate(Component = "Transportation")
health_df <- get_cansim_vector(cpi_vectors["Health_personal_care"], start_time = "2000-01-01") %>% mutate(Component = "Health_personal_care")
recreation_df <- get_cansim_vector(cpi_vectors["Recreation_education"], start_time = "2000-01-01") %>% mutate(Component = "Recreation_education")
alcohol_df <- get_cansim_vector(cpi_vectors["Alcohol_tobacco"], start_time = "2000-01-01") %>% mutate(Component = "Alcohol_tobacco")


Warning message in as.POSIXlt.POSIXct(x, tz = tz):
"unknown timezone 'America/Toronto'"


Warning message in as.POSIXlt.POSIXct(x, tz = tz):
"unknown timezone 'GMT'"
Warning message in as.POSIXlt.POSIXct(x, tz = tz):
"unknown timezone 'America/New_York'"
Warning message in as.POSIXlt.POSIXct(x, tz = tz):
"unknown timezone 'America/Los_Angeles'"
Warning message in as.POSIXlt.POSIXct(x, tz = tz):
"unknown timezone 'GMT'"
Warning message in as.POSIXlt.POSIXct(x, tz = tz):
"unknown timezone 'America/New_York'"
Reading CANSIM NDM vectors from temporary cache

Warning message in as.POSIXlt.POSIXct(x, tz = tz):
"unknown timezone 'America/Toronto'"
Warning message in as.POSIXlt.POSIXct(x, tz = tz):
"unknown timezone 'GMT'"
Warning message in as.POSIXlt.POSIXct(x, tz = tz):
"unknown timezone 'America/New_York'"
Warning message in as.POSIXlt.POSIXct(x, tz = tz):
"unknown timezone 'America/Los_Angeles'"
Warning message in as.POSIXlt.POSIXct(x, tz = tz):
"unknown timezone 'GMT'"
Warning message in as.POSIXlt.POSIXct(x, tz = tz):
"unknown timezone 'America/New_York'"
Reading CANSIM NDM 

In [9]:
raw_data_list <- rbind(food_df, shelter_df, household_df, clothing_df, transport_df, health_df, recreation_df, alcohol_df)

cat("Download complete. Processing data...\n")

# Clean the data and pivot to a wide format
cpi_data_wide <- raw_data_list %>%
  dplyr::select(Date = REF_DATE, Component, Value = VALUE) %>%
  # Pivot so each component has its own column
  pivot_wider(names_from = Component, values_from = Value) %>%
  # Arrange by date to ensure correct order
  arrange(Date)

cat("Successfully processed CPI data. Preview:\n")
print(head(cpi_data_wide))


Download complete. Processing data...


Successfully processed CPI data. Preview:
# A tibble: 6 × 9
  Date        Food Shelter Household_operations Clothing_footwear Transportation
  <chr>      <dbl>   <dbl>                <dbl>             <dbl>          <dbl>
1 2000-01-01  91.8    93.8                 95.8             100.            94.7
2 2000-02-01  91.8    94.4                 96.1             100.            95.6
3 2000-03-01  92.1    94.5                 96.3              99.9           97.5
4 2000-04-01  92.3    94.5                 96.1              99.5           96.1
5 2000-05-01  92.8    94.7                 96.2             100.            96.6
6 2000-06-01  93.1    95                   96.5             100.            98  
# ℹ 3 more variables: Health_personal_care <dbl>, Recreation_education <dbl>,
#   Alcohol_tobacco <dbl>


In [ ]:
groups <- list(
  Shelter = c("Shelter"),
  Volatile_Components = c("Food", "Transportation"),
  Core_Goods = c("Household_operations", "Clothing_footwear", "Alcohol_tobacco"),
  Core_Services = c("Health_personal_care", "Recreation_education")
)


In [53]:
calculate_group_index <- function(cpi_values, weights) {
  
  # Sum the weights for only the components in the group for a given month
  total_group_weight <- sum(weights, na.rm = TRUE)
  
  # Avoid division by zero if weights are missing or zero
  if (total_group_weight == 0) return(NA_real_)
  
  # Explicitly rescale the weights so they sum to 1 *within the group*
  rescaled_weights <- weights / total_group_weight
  
  # Calculate the final index by summing the product of CPI values and rescaled weights
  weighted_index <- sum(cpi_values * rescaled_weights, na.rm = TRUE)
  
  return(weighted_index)
}


# 2. Apply this function to your joined data
# This code replaces the final calculation step from the previous script
final_weighted_cpi_levels_explicit <- data_with_weights %>%
  # Perform the operation for each row (i.e., for each month)
  rowwise() %>%
  mutate(
    Shelter_Index = calculate_group_index(
      cpi_values = c_across(all_of(paste0(groups$Shelter, ".x"))),
      weights    = c_across(all_of(paste0(groups$Shelter, ".y")))
    ),
    Volatile_Components_Index = calculate_group_index(
      cpi_values = c_across(all_of(paste0(groups$Volatile_Components, ".x"))),
      weights    = c_across(all_of(paste0(groups$Volatile_Components, ".y")))
    ),
    Core_Goods_Index = calculate_group_index(
      cpi_values = c_across(all_of(paste0(groups$Core_Goods, ".x"))),
      weights    = c_across(all_of(paste0(groups$Core_Goods, ".y")))
    ),
    Core_Services_Index = calculate_group_index(
      cpi_values = c_across(all_of(paste0(groups$Core_Services, ".x"))),
      weights    = c_across(all_of(paste0(groups$Core_Services, ".y")))
    )
  ) %>%
  ungroup() %>%
  # Select the final columns you need
  dplyr::select(Date, ends_with("_Index"))

# View the result
cat("Successfully calculated dynamically weighted CPI levels with explicit rescaling. Preview:\n")
glimpse(final_weighted_cpi_levels_explicit)


Successfully calculated dynamically weighted CPI levels with explicit rescaling. Preview:
Rows: 307
Columns: 5
$ Date                      <chr> "2000-01-01", "2000-02-01", "2000-03-01", "2…
$ Shelter_Index             <dbl> NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, …
$ Volatile_Components_Index <dbl> NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, …
$ Core_Goods_Index          <dbl> NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, …
$ Core_Services_Index       <dbl> NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, …


In [35]:
cpi_yoy_changes <- cpi_data_wide %>%
  # Arrange by date to ensure the 12-month lag is calculated correctly
  arrange(Date) %>%
  mutate(across(
    .cols = -Date, # Apply to all columns except 'Date'
    # Change lag from 1 to 12
    .fns = ~ (.x / lag(.x, 12) - 1) * 100, 
    .names = "yoy_{.col}" # Create new columns with a 'yoy_' prefix
  )) %>%
  # Select only the new yoy columns and the Date column
  dplyr::select(Date, starts_with("yoy_")) %>%
  # Remove the first 12 rows which will be NA after the lag() operation
  na.omit()

cat("Successfully calculated monthly changes. Preview:\n")
print(head(cpi_yoy_changes))

Successfully calculated monthly changes. Preview:
# A tibble: 6 × 9
  Date       yoy_Food yoy_Shelter yoy_Household_operations yoy_Clothing_footwear
  <chr>         <dbl>       <dbl>                    <dbl>                 <dbl>
1 2001-01-01     3.49        3.94                     1.67                 0.598
2 2001-02-01     4.47        3.18                     1.56                 1.10 
3 2001-03-01     4.89        4.02                     1.45                 1.20 
4 2001-04-01     4.98        4.66                     2.19                 1.31 
5 2001-05-01     5.06        5.60                     2.39                 0.299
6 2001-06-01     4.73        5.58                     2.28                 0    
# ℹ 4 more variables: yoy_Transportation <dbl>, yoy_Health_personal_care <dbl>,
#   yoy_Recreation_education <dbl>, yoy_Alcohol_tobacco <dbl>


In [38]:
# Prepare the CPI data by adding a 'Year' column
cpi_to_join <- cpi_yoy_changes %>%
  mutate(Year = year(Date))

# Prepare the weights data by adding a 'Year' column
weights_to_join <- yearly_weights_df %>%
  mutate(Year = year(date)) %>%
  dplyr::select(-date) # Remove the old date column to avoid confusion

# Join the two dataframes by 'Year'
final_historical_df <- cpi_to_join %>%
  left_join(weights_to_join, by = "Year")

cat("Successfully joined CPI changes with historical weights. Preview:\n")
print(head(final_historical_df))

Successfully joined CPI changes with historical weights. Preview:
# A tibble: 6 × 21
  Date       yoy_Food yoy_Shelter yoy_Household_operations yoy_Clothing_footwear
  <chr>         <dbl>       <dbl>                    <dbl>                 <dbl>
1 2001-01-01     3.49        3.94                     1.67                 0.598
2 2001-02-01     4.47        3.18                     1.56                 1.10 
3 2001-03-01     4.89        4.02                     1.45                 1.20 
4 2001-04-01     4.98        4.66                     2.19                 1.31 
5 2001-05-01     5.06        5.60                     2.39                 0.299
6 2001-06-01     4.73        5.58                     2.28                 0    
# ℹ 16 more variables: yoy_Transportation <dbl>,
#   yoy_Health_personal_care <dbl>, yoy_Recreation_education <dbl>,
#   yoy_Alcohol_tobacco <dbl>, Year <dbl>, Food <dbl>, Shelter <dbl>,
#   Household_operations <dbl>, Clothing_footwear <dbl>, Transportation <dbl>,
#

In [10]:
groups <- list(
  Shelter = c("Shelter"),
  Volatile_Components = c("Food", "Transportation"),
  Core_Goods = c("Household_operations", "Clothing_footwear", "Alcohol_tobacco"),
  Core_Services = c("Health_personal_care", "Recreation_education")
)


In [12]:
final_weighted_cpi <- cpi_data_wide %>% dplyr::select(Date)

# Loop through each defined group to calculate its weighted CPI
for (group_name in names(groups)) {
  
  # Get the component names for the current group
  components_in_group <- groups[[group_name]]
  
  # Get the CPI data and weights for only these components
  group_cpi_data <- cpi_data_wide[, components_in_group, drop = FALSE]
  group_weights <- cpi_weights[components_in_group]
  
  # Calculate the sum of weights for this specific group
  total_group_weight <- sum(group_weights)
  
  # Re-scale the weights so they sum to 1 within the group
  rescaled_weights <- group_weights / total_group_weight
  
  # Calculate the weighted average for the group
  # We use matrix multiplication for an efficient calculation across all dates
  weighted_cpi_series <- as.matrix(group_cpi_data) %*% as.numeric(rescaled_weights)
  
  # Add the new calculated series to our final data frame
  final_weighted_cpi[[group_name]] <- weighted_cpi_series
}


In [13]:
print(head(final_weighted_cpi))

# A tibble: 6 × 5
  Date       Shelter[,1] Volatile_Components[…¹ Core_Goods[,1] Core_Services[,1]
  <chr>            <dbl>                  <dbl>          <dbl>             <dbl>
1 2000-01-01        93.8                   93.2           93.1              96.0
2 2000-02-01        94.4                   93.7           93.2              96.1
3 2000-03-01        94.5                   94.8           93.4              96.4
4 2000-04-01        94.5                   94.2           93.2              96.4
5 2000-05-01        94.7                   94.7           93.4              96.6
6 2000-06-01        95                     95.5           93.7              96.9
# ℹ abbreviated name: ¹​Volatile_Components[,1]


In [14]:
print(tail(final_weighted_cpi))


# A tibble: 6 × 5
  Date       Shelter[,1] Volatile_Components[…¹ Core_Goods[,1] Core_Services[,1]
  <chr>            <dbl>                  <dbl>          <dbl>             <dbl>
1 2025-02-01        187.                   183.           137.              138.
2 2025-03-01        187                    183.           138.              137.
3 2025-04-01        187.                   183.           138.              137.
4 2025-05-01        187.                   183.           139.              137.
5 2025-06-01        187.                   183.           139.              138.
6 2025-07-01        188.                   184.           139.              137.
# ℹ abbreviated name: ¹​Volatile_Components[,1]


In [15]:

### EXCHANGE RATE US-CANADA ###
exc_rate <- fredr(
  series_id = "DEXCAUS",
  observation_start = as.Date("1990-01-01"),
  observation_end = as.Date("2025-08-01"),
  frequency = "m", # monthly
  units = "chg" # change over previous value
)

### TRADE COMMODITIES US-CANADA ###
trade_commodities <- fredr(
  series_id = "XTNTVA01CAM664S",
  observation_start = as.Date("1990-01-01"),
  observation_end = as.Date("2025-08-01"),
  frequency = "m", # monthly
  units = "chg" # change over previous value
)


### TRADE POLICY UNCERTAINTY INDEX ###
tpu_data <- read_csv("Categorical_EPU_Data.csv")|>
mutate(Date = ymd(paste(Year, Month, "01", sep = "-")))|>
rename(VALUE = '9. Trade policy')|>
glimpse()


### lFS MICRO ###
lfs_micro <- read_csv('C:/Users/Irene/governorschallenge/INDINF_WAGES_MONTHLY.csv', skip=20)|>
dplyr::select(date, INDINF_LFSMICRO_M)|>
rename(Date = date, VALUE = INDINF_LFSMICRO_M)|>
  mutate(Date = mdy(Date)) |>
  filter(Date > ymd("2000-12-31"))|>
glimpse()


unemp_vec <- get_cansim_vector_for_latest_periods('v2062815', periods = 500, refresh = TRUE) # UNEMPLOYMENT RATE
fcpi_vec <- get_cansim_vector_for_latest_periods('v52673496', periods = 500, refresh = TRUE) ## FISHER COMMODITY PRICE INDEX
wti_price_monthly <- fredr::fredr(series_id = "MCOILWTICO") ## CRUDE OUL PRICES -- WEST TEXAS INTERMEDIATE


Rows: 488 Columns: 3
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (1): Year
dbl (2): Month, 9. Trade policy

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Warning message:
"There was 1 warning in `mutate()`.
ℹ In argument: `Date = ymd(paste(Year, Month, "01", sep = "-"))`.
Caused by warning:
!  1 failed to parse."


Rows: 488
Columns: 4
$ Year  <chr> "1985", "1985", "1985", "1985", "1985", "1985", "1985", "1985", …
$ Month <dbl> 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 1, 2, 3, 4, 5, 6, 7, 8, 9…
$ VALUE <dbl> 122.61396, 41.76222, 107.92573, 194.75287, 124.39166, 153.16361,…
$ Date  <date> 1985-01-01, 1985-02-01, 1985-03-01, 1985-04-01, 1985-05-01, 198…


Rows: 390 Columns: 8
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (1): date
dbl (7): INDINF_LFSMICRO_M, INDINF_LFSMICRO_LEVELS_M, INDINF_LFSWC_M, INDINF...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


Rows: 294
Columns: 2
$ Date  <date> 2001-01-01, 2001-02-01, 2001-03-01, 2001-04-01, 2001-05-01, 200…
$ VALUE <dbl> 2.8, 3.3, 3.4, 3.3, 3.6, 3.3, 3.3, 3.1, 3.0, 3.0, 3.2, 3.4, 3.2,…


Accessing CANSIM NDM vectors from Statistics Canada

Accessing CANSIM NDM vectors from Statistics Canada



In [ ]:
## BREAK EVEN INFLATION RATE

monthly_real_return_bond <- get_cansim_vector_for_latest_periods('v122553',periods = 500)
monthly_convetional_bond <- get_cansim_vector_for_latest_periods('v122543',periods = 500)
monthly_real_return_bond <- monthly_real_return_bond|>
dplyr::select(Date, VALUE)

monthly_convetional_bond <- monthly_convetional_bond|>
dplyr::select(Date, VALUE)

break_even_df <- monthly_convetional_bond |>
  inner_join(monthly_real_return_bond, by = "Date", suffix = c("_conventional", "_real_return")) |>
  mutate(break_even_inflation_rate = VALUE_conventional - VALUE_real_return) |>
  dplyr::select(Date, break_even_inflation_rate)

  glimpse(
break_even_df)

In [ ]:
supply_chain_pressure <- read_csv("C:/Users/Irene/governorschallenge/gscpi_data.csv")|>
dplyr::select(Date, GSCPI)





New names:
• `` -> `...3`
• `` -> `...4`
• `` -> `...5`
• `` -> `...6`
• `` -> `...7`
• `` -> `...8`
• `` -> `...9`
• `` -> `...10`
• `` -> `...11`
• `` -> `...12`
• `` -> `...13`
• `` -> `...14`
Rows: 331 Columns: 14
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr  (1): Date
dbl  (1): GSCPI
lgl (12): ...3, ...4, ...5, ...6, ...7, ...8, ...9, ...10, ...11, ...12, ......

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
